In [5]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd
import joblib

# Load the dataset
data = pd.read_csv('user_dataset_train_v2.csv')

# Perform data cleaning if needed (e.g., removing duplicates)
data = data.drop_duplicates()

# Define features (X) and target variable (y)
X = data[['is_email_verified', 'is_phone_verified', 'city', 'gender', 'timezone', 'user_agent', 'device_id', 'ip_address']]
y = data['is_abuser']

# Define categorical features for OneHotEncoding
categorical_features = ['city', 'gender', 'timezone']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create a preprocessor to apply transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle data imbalance using RandomOverSampler
X_train_resampled, y_train_resampled = RandomOverSampler(sampling_strategy=0.1).fit_resample(X_train, y_train)
# print(pd.Series(y_train_resampled).value_counts())

# Initialize the RandomForestClassifier
model = RandomForestClassifier()

# Create a pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', model)])

# Train the model
pipeline.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Generate a classification report
print(classification_report(y_test, y_pred))

#Save the model for deployment
joblib.dump(model, 'promo_code_abuse_model.pkl')

Accuracy: 0.9611111111111111
              precision    recall  f1-score   support

       False       0.98      0.98      0.98      1765
        True       0.00      0.00      0.00        35

    accuracy                           0.96      1800
   macro avg       0.49      0.49      0.49      1800
weighted avg       0.96      0.96      0.96      1800



['promo_code_abuse_model.pkl']

In [3]:
# Assuming 'X_test_new' is your new testing data
X_test_new = pd.read_csv('user_dataset_test_v2.csv')

y_test_new = X_test_new['is_abuser']
# Drop the 'is_abuser' column from X_test_new
X_test_new = X_test_new.drop(columns=['is_abuser'])

# Make predictions on the new testing data
y_pred_new = pipeline.predict(X_test_new)

X_test_new['model_output'] = y_pred_new
X_test_new['is_abuser'] = y_test_new
X_test_new.to_csv('user_dataset_test_with_predictions.csv', index=False)
# Compare the predicted values with the actual values in the 'is_abuser' column
# Assuming 'y_test_new' is the actual 'is_abuser' column in your new testing data
# accuracy_new = accuracy_score(y_test_new, y_pred_new)
# print(f'Accuracy on new data: {accuracy_new}')

# Generate a classification report for the new data
# print(classification_report(y_test_new, y_pred_new))
